## Notebook for adding a train or test column to rows in a dataframe of wells loaded but not features added yet

In [10]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
import pickle
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client
import random 

In [11]:
print(welly.__version__)
print(dask.__version__)
print(pd.__version__)

0.3.5
0.18.2
0.23.3


In [12]:
%%timeit
import os
env = %env

83.6 µs ± 2.37 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
#### Had to change display options to get this to print in full!
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000

In [14]:
knn_dir = "../WellsKNN/"
load_dir = "../loadLAS/"
features_dir = "../createFeatures/"
check_dir = "../CheckAvailableData/"
loadLas_dir = "../loadLAS/"


## File with wells loaded but not features or train test split yet

In [15]:
file_with_wells = 'df_all_wells_noKNNorFeatures_20180927_a.h5'

## File to write to at end = 

In [16]:
h5_filename_df_wells_wTrainTestSplitCol = 'df_all_wells_noKNNorFeatures_wTrainSplit_20180927_c.h5'

## Split variables

In [17]:
split_variable = 0.8

## Import all the data

In [18]:
##df_1.to_hdf('df_all_wells_noKNNorFeatures_20180920_a.h5', key='df', mode='w')
df_all_Col_preSplit = pd.read_hdf(load_dir+file_with_wells, 'df')

## Split data into train & test on a well by well basis, not row basis

In [19]:
UWIs = list(df_all_Col_preSplit['UWI'].unique())

In [20]:
numberOfTrainingWells = math.floor(len(UWIs)*split_variable)
numberOfTrainingWells

1280

Randomly select that number of UWIs for training and the ones left for test

In [21]:
UWIs_training = random.sample(UWIs, numberOfTrainingWells)

In [22]:
UWIs_test = [x for x in UWIs if x not in UWIs_training]

In [23]:
print("train",len(UWIs_training))
print("test",len(UWIs_test))

train 1280
test 321


In [24]:
df_all_Col_preSplit_wTrainTest = df_all_Col_preSplit.copy()

In [25]:
df_all_Col_preSplit_wTrainTest['trainOrTest'] = np.where(df_all_Col_preSplit_wTrainTest['UWI'].isin(UWIs_training), 'train', 'test')

In [26]:
df_all_Col_preSplit_wTrainTest.tail()

,CALI,COND,DELT,DEPT,DPHI,DT,GR,ILD,ILM,NPHI,PHID,RHOB,SFL,SFLU,SN,SP,UWI,trainOrTest
876,NaN,NaN,NaN,359.00,0.014,NaN,61.724,53.940,NaN,0.191,NaN,NaN,NaN,NaN,NaN,NaN,00-10-35-081-15W4-0.LAS,train
877,NaN,NaN,NaN,359.25,0.014,NaN,59.927,63.882,NaN,0.167,NaN,NaN,NaN,NaN,NaN,NaN,00-10-35-081-15W4-0.LAS,train
878,NaN,NaN,NaN,359.50,0.011,NaN,58.729,74.245,NaN,0.155,NaN,NaN,NaN,NaN,NaN,NaN,00-10-35-081-15W4-0.LAS,train
879,NaN,NaN,NaN,359.75,0.007,NaN,57.529,93.046,NaN,0.148,NaN,NaN,NaN,NaN,NaN,NaN,00-10-35-081-15W4-0.LAS,train
880,NaN,NaN,NaN,360.00,0.006,NaN,56.926,138.167,NaN,0.140,NaN,NaN,NaN,NaN,NaN,NaN,00-10-35-081-15W4-0.LAS,train


## Save to file

In [27]:
df_all_Col_preSplit_wTrainTest.to_hdf(h5_filename_df_wells_wTrainTestSplitCol, key='df', mode='w')